#Rodar experimento

In [2]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
tokenizer = AutoTokenizer.from_pretrained('pablocosta/bertabaporu-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/504k [00:00<?, ?B/s]

In [3]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/Trabalho-webmidia/csv/grupos_discordancia.csv')
df = pd.read_csv('/content/drive/MyDrive/grupos_discordancia.csv')
df = df[df['class_result'] != 10]
df = df[df['class_result'] != -10]
len(df)
df['class_result'] = [x + 1 for x in list(df['class_result'])]

In [8]:
import re
import html
def pre_processamento (x):

  retorno = re.sub(r'http(s)?://\S+|www\.\S+', '', x)
  verdadeiro = re.search(r'&\S+', retorno)

  if verdadeiro:
        retorno = html.unescape(retorno)

  return retorno

In [9]:
df.loc[:, 'Comentário'] = df.loc[:, 'Comentário'].apply(lambda x: pre_processamento(x))

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
import torch

In [13]:
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [14]:
MAX_LENGTH = 180
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
RANDOM_SEED = 42

In [15]:
#df_teste_separado = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')
#df_teste_separado = pd.read_csv('/content/drive/MyDrive/resultado_consolidado.csv')
df_teste_separado = pd.read_csv('/content/drive/MyDrive/resultado_modelos.csv')

In [16]:
lista_ids = (df_teste_separado['id'])

In [17]:
df_kfold = df
df_kfold = df_kfold[df_kfold['id'].isin(lista_ids) == False]
len(df_kfold)

1535

In [18]:
class BertDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, index):
    review = str(self.reviews[index])
    target = self.targets[index]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      #padding='longest',
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
      truncation=True
    )

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [19]:
df_data = BertDataset(reviews=df_kfold['Comentário'].to_numpy(),
                            targets=df_kfold['class_result'].to_numpy(),
                            tokenizer=tokenizer,
                            max_len=MAX_LENGTH)

df_teste = BertDataset(reviews=df_teste_separado['Comentário'].to_numpy(),
                            targets=df_teste_separado['class_result'].to_numpy(),
                            tokenizer=tokenizer,
                            max_len=MAX_LENGTH)

In [20]:
from transformers import AutoModelForSequenceClassification

In [21]:
import transformers

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [23]:
from sklearn.metrics import precision_recall_fscore_support

In [24]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [25]:
k_folds = 10
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

In [26]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
from sklearn.metrics import accuracy_score

In [27]:
def train_model(model, data_loader, criterian, optimizer, scheduler, n_examples):

    model.train()
    train_loss = []
    correct_pred = 0
    all_predictions = []
    all_targets = []

    for d in data_loader:

        input_ids = d['input_ids'].to(device)
        attention_masks = d['attention_mask'].to(device)
        targets = d['targets'].to(device)

        # forward prop
        predictions = model(input_ids , attention_masks)
        loss = criterian(predictions.logits, targets)

        _,pred_classes = torch.max(predictions.logits, dim=1)
        #backprop
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        #scheduler.step()
        optimizer.zero_grad()
        train_loss.append(loss.item())

        correct_pred += torch.sum(pred_classes==targets)
        all_predictions.extend(pred_classes.cpu().numpy()) #extend significa que vai colocar vários itens de uma vez
        #o pred_classes está no gpu, mas a função que calcula o f1_score precisa que esteja no cpu
        all_targets.extend(targets.cpu().numpy())

    precision, recall, f1_score, _ = precision_recall_fscore_support(
        all_targets, all_predictions, average='weighted', zero_division=1
    ) #o wighted serve para levar em conta a discrepância entre o quantidade de cada um dos 3 rótulos

    acc = accuracy_score(all_targets, all_predictions)

    return acc, precision, recall, f1_score, np.mean(train_loss)

In [28]:
def eval_model(model, data_loader, criterian, n_examples):
    model.eval()
    eval_loss = []
    correct_pred = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for d in data_loader:

            input_ids = d['input_ids'].to(device)
            attention_masks = d['attention_mask'].to(device)
            targets = d['targets'].to(device)

            # forward prop
            predictions = model(input_ids , attention_masks)
            loss = criterian(predictions.logits, targets)
            _,pred_classes = torch.max(predictions.logits, dim=1)
            eval_loss.append(loss.item())

            correct_pred += torch.sum(pred_classes==targets)
            all_predictions.extend(pred_classes.cpu().numpy()) #extend significa que vai colocar vários itens de uma vez
            #o pred_classes está no gpu, mas a função que calcula o f1_score precisa que esteja no cpu
            all_targets.extend(targets.cpu().numpy())

    precision, recall, f1_score, _ = precision_recall_fscore_support(
        all_targets, all_predictions, average='weighted', zero_division=1
    ) #o wighted serve para levar em conta a discrepância entre o quantidade de cada um dos 3 rótulos

    acc = accuracy_score(all_targets, all_predictions)

    return acc, precision, recall, f1_score, np.mean(eval_loss)

In [29]:
class EarlyStopper:
    def __init__(self, patience, min_delta):
        self.patience = patience #número épocas que serviram como limite para a perda voltar a diminuir
        self.min_delta = min_delta #número minímo que considera que houve diminuição
        self.counter = 0 #contador
        self.min_validation_loss = float('inf') #menor valor de perda de validação, iniciado com infinito

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
import sklearn
print(sklearn.__version__)

1.6.0


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)


fold_accuracies = []
fold_f1_score = []
fold_recall = []
fold_precision = []


for fold, (train_index, test_index) in enumerate(kf.split(df_kfold)): #contador(fold) e indices que definem os comentários de treinamento e validação
    print(f"Training Fold {fold+1}/{k_folds}")

    model = AutoModelForSequenceClassification.from_pretrained('pablocosta/bertabaporu-base-uncased', num_labels=3).to(device)
    # Split dataset into train and validation sets for the current fold
    train_dataset = torch.utils.data.Subset(df_data, train_index)
    val_dataset = torch.utils.data.Subset(df_data, test_index)
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Training loop
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    criterion = torch.nn.CrossEntropyLoss().to(device)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    acuracia_val = None
    f1_score_val = None
    precisao_val = None
    recall_val = None
    loss_val = None

    acuracia_val_antigo = None
    precisao_val_antigo = None
    recall_val_antigo = None
    f1_score_val_antigo = None
    loss_val_antigo = None

    model.train()

    verificar_parada = EarlyStopper(1, 0.001)

    melhor_model = [] #salva o melhor modelo do epoch

    for epoch in range(50):
        #treinamento
        train_acc, train_precision, train_recall, train_f1_score, train_loss = train_model(model,
                                         train_loader,
                                         criterion,
                                         optimizer,
                                         None,
                                         len(train_loader))

        #validação
        acuracia_val, precisao_val, recall_val, f1_score_val, loss_val = eval_model(model,
                                    val_loader,
                                    criterion,
                                    len(val_loader))

        if verificar_parada.early_stop(loss_val):
          print()
          print(f"loss antiga: {loss_val_antigo}, loss atual: {loss_val}" )
          torch.save(melhor_model[0], melhor_model[1])
          print(f"MODELO_{fold} Accuracy: {acuracia_val_antigo}, F1_Score: {f1_score_val_antigo}, recall_Val: {recall_val_antigo}, precisao: {precisao_val_antigo} loss: {loss_val_antigo}")
          print()
          break

        melhor_model = []
        melhor_model.append(model.state_dict())
        melhor_model.append(f'/content/drive/MyDrive/modelos_abaporu/model_state_{fold}.bin')


        acuracia_val_antigo = acuracia_val
        precisao_val_antigo = precisao_val
        recall_val_antigo = recall_val
        f1_score_val_antigo = f1_score_val
        loss_val_antigo = loss_val

    fold_accuracies.append(acuracia_val_antigo)
    fold_f1_score.append(f1_score_val_antigo)
    fold_recall.append(recall_val_antigo)
    fold_precision.append(precisao_val_antigo)


# Calculate average accuracy across all folds
average_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print(f"MEDIAN => Accuracy: {average_accuracy} F1_score: {sum(fold_f1_score)/len(fold_f1_score)} recall: {sum(fold_recall)/len(fold_recall)} precision: {sum(fold_precision)/len(fold_precision)}")

Training Fold 1/10


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



loss antiga: 0.5751299619674682, loss atual: 0.6514270961284637
MODELO_0 Accuracy: 0.7857142857142857, F1_Score: 0.788215092258219, recall_Val: 0.7857142857142857, precisao: 0.7938494588118649 loss: 0.5751299619674682

Training Fold 2/10


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...


loss antiga: 0.5452387154102325, loss atual: 0.5576234817504883
MODELO_7 Accuracy: 0.7581699346405228, F1_Score: 0.7546151762832349, recall_Val: 0.7581699346405228, precisao: 0.7610125718454198 loss: 0.5452387154102325

Training Fold 9/10


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



loss antiga: 0.7696737170219421, loss atual: 0.7868115186691285
MODELO_8 Accuracy: 0.6601307189542484, F1_Score: 0.6412377988031914, recall_Val: 0.6601307189542484, precisao: 0.6427510398098634 loss: 0.7696737170219421

Training Fold 10/10


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



loss antiga: 0.6983612298965454, loss atual: 0.725445032119751
MODELO_9 Accuracy: 0.7254901960784313, F1_Score: 0.6962277979836566, recall_Val: 0.7254901960784313, precisao: 0.7354489164086687 loss: 0.6983612298965454

MEDIAN => Accuracy: 0.7322043969102794 F1_score: 0.7227110662741589 recall: 0.7322043969102794 precision: 0.7300182875192861


In [30]:
num_classes = 3


model_name = 'pablocosta/bertabaporu-base-uncased'

In [31]:
def get_predictions(model, data_loader):
    model = model.eval()

    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:

            texts = d["review_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"]

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask)

            _, preds = torch.max(outputs.logits, dim=1)

            probs = F.softmax(outputs.logits, dim=1)

            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values

In [32]:
BATCH_SIZE = 32
test_loader = DataLoader(df_teste, batch_size=BATCH_SIZE, shuffle=True) #shuffle=True evirar overfitting

In [33]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
from sklearn.metrics import accuracy_score

In [34]:
from tqdm import tqdm
import torch.nn.functional as F

fold_accuracies = []
fold_f1_score = []
fold_recall = []
fold_precision = []

for i in range(10):
  best_model_path = f'/content/drive/MyDrive/modelos_abaporu/model_state_{i}.bin'
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes).to(device)
  model.load_state_dict(torch.load(best_model_path))
  y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model,test_loader)
  acc_t = accuracy_score(y_test, y_pred)
  f1_score_t = f1_score(y_test, y_pred, average='weighted')
  precisao_t = precision_score(y_test, y_pred, average='weighted')
  recall_t = recall_score(y_test, y_pred, average='weighted')
  print()
  print(f"modelo_{i}  acc: {acc_t}, f1_score: {f1_score_t}, precisao: {precisao_t}, recall: {recall_t}")
  print()
  fold_accuracies.append(acc_t)
  fold_f1_score.append(f1_score_t)
  fold_precision.append(precisao_t)
  fold_recall.append(recall_t)
print(f"MEDIAN => Accuracy: {sum(fold_accuracies)/len(fold_accuracies)} F1_score: {sum(fold_f1_score)/len(fold_f1_score)} recall: {sum(fold_recall)/len(fold_recall)} precision: {sum(fold_precision)/len(fold_precision)}")

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-34-9753171dc85e>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_saf


modelo_0  acc: 0.7777777777777778, f1_score: 0.772380733715122, precisao: 0.7745238250709853, recall: 0.7777777777777778



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-34-9753171dc85e>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_saf


modelo_1  acc: 0.7777777777777778, f1_score: 0.7751576414933271, precisao: 0.7864290815046417, recall: 0.7777777777777778



...


modelo_7  acc: 0.7953216374269005, f1_score: 0.7938867849341975, precisao: 0.7970058124033489, recall: 0.7953216374269005



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-34-9753171dc85e>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_saf


modelo_8  acc: 0.7660818713450293, f1_score: 0.7583387557970516, precisao: 0.7689393525350239, recall: 0.7660818713450293



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-34-9753171dc85e>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_saf


modelo_9  acc: 0.7309941520467836, f1_score: 0.7282220598402519, precisao: 0.7299816808555422, recall: 0.7309941520467836

MEDIAN => Accuracy: 0.7631578947368423 F1_score: 0.7589458362464121 recall: 0.7631578947368423 precision: 0.7661848453752483


**MELHOR MODELO: MODELO_7**

In [35]:

best_model_path = f'/content/drive/MyDrive/modelos_abaporu/model_state_{7}.bin'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes).to(device)
model.load_state_dict(torch.load(best_model_path))
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model,test_loader)
acc_t = accuracy_score(y_test, y_pred)
f1_score_t = f1_score(y_test, y_pred, average='weighted')
precisao_t = precision_score(y_test, y_pred, average='weighted')
recall_t = recall_score(y_test, y_pred, average='weighted')
print()
print(f"modelo_{7}  acc: {acc_t}, f1_score: {f1_score_t}, precisao: {precisao_t}, recall: {recall_t}")
print()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pablocosta/bertabaporu-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-35-a9c6224c08f8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe


modelo_7  acc: 0.7953216374269005, f1_score: 0.7938867849341975, precisao: 0.7970058124033489, recall: 0.7953216374269005



In [ ]:
df_consolidados = pd.read_csv('/content/drive/MyDrive/resultado_consolidado.csv')

In [ ]:
df_consolidados

,Unnamed: 0,id,Comentário,class_result,total_agreement,gpt_prompt1,gpt_prompt2,abaporu,bertimbau
0,0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1,2,2,2
1,1,hub93yq,Campeonato de quarto,1,1,1,1,1,1
2,2,hv1xsjp,Não entendi,1,1,1,1,1,1
3,3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0,0,0,0
4,4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...
166,166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1,1,1,0
167,167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2,2,2,2
168,168,izqc5yj,Lógico que desabou kkkkk,1,0,1,1,0,1
169,169,j0kxoll,como é bom ter democracia,2,1,2,2,2,2


In [ ]:
dic_test = {'Comentário': y_review_texts, 'abaporu': y_pred}
df_dic_test = pd.DataFrame(dic_test)
df_dic_test

,Comentário,abaporu
0,#fora dorival volta pro vozão,0
1,Espera! Hummm não! Senti só vergonha mesmo por...,0
2,Por que foi.,1
3,Esses bodybuilders morrem do coração por decor...,0
4,Professor de economia falando esquerdista reta...,0
...,...,...
166,Powered by MS Paint™,1
167,Cada dia que passa eu tenho mais certeza que é...,0
168,Porque na cabeça dela o simples fato dela ser ...,1
169,No meio desses 2.7 anos John chutou uma pedra ...,1


In [ ]:
merge = pd.merge(df_teste_separado, df_dic_test, how='inner' ,on='Comentário')
merge.drop('Unnamed: 0.1', axis=1, inplace=True)
merge.drop('Unnamed: 0', axis=1, inplace=True)
merge

,id,Comentário,class_result,total_agreement,gpt_prompt1,gpt_prompt2,abaporu
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1,2,2
1,hub93yq,Campeonato de quarto,1,1,1,1,1
2,hv1xsjp,Não entendi,1,1,1,1,1
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0,0,0
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1,1,1
...,...,...,...,...,...,...,...
166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1,1,1
167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2,2,2
168,izqc5yj,Lógico que desabou kkkkk,1,0,1,1,0
169,j0kxoll,como é bom ter democracia,2,1,2,2,2


In [ ]:
merge.to_csv('/content/drive/MyDrive/resultado_consolidado.csv')